In [ ]:
import numpy as np
from scipy import stats
from entropy import entropy
from pywt import wavedec
from scipy.signal import welch


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:


# Assuming you have EEG data in epochs (shape: [n_epochs, n_channels, n_samples])
features = []

for epoch_data in epochs:
    # Average Power
    avg_power = np.mean(epoch_data ** 2, axis=1)

    # Root Mean Square
    rms = np.sqrt(np.mean(epoch_data ** 2, axis=1))

    # Standard Deviation
    std_dev = np.std(epoch_data, axis=1)

    # Variance
    variance = np.var(epoch_data, axis=1)

    # Entropy
    entropy_values = []
    for channel_data in epoch_data:
        entropy_values.append(entropy(channel_data))

    # Discrete Wavelet Transform (DWT)
    dwt_coefficients = []
    for channel_data in epoch_data:
        coeffs = wavedec(channel_data, 'db4', level=4)  # Adjust wavelet and level as needed
        dwt_coefficients.append(np.hstack(coeffs))

    # Power Spectral Density (PSD)
    freqs, psd = welch(epoch_data, fs=epochs.info['sfreq'], nperseg=256)  # Adjust nperseg as needed

    # Combine all features for this epoch
    epoch_features = np.hstack([avg_power, rms, std_dev, variance, entropy_values, np.array(dwt_coefficients).flatten(), psd.flatten()])

    features.append(epoch_features)

features = np.array(features)


In [ ]:


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
